# RAG Query Pipeline
## 1. Imports thư viện cần thiết

In [41]:
# pip install langchain-groq
import os
import chromadb
from langchain_groq import ChatGroq
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

print("✅ Đã import thành công các thư viện cần thiết cho RAG pipeline.")

✅ Đã import thành công các thư viện cần thiết cho RAG pipeline.


## 2. Khởi tạo Mô hình (LLM và Embedding)

In [42]:
# Cấu hình biến môi trường cho Groq API Key trong file .env
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
print("✅ Đã cấu hình biến môi trường GROQ_API_KEY từ file .env.")

model_llm = "llama-3.1-8b-instant"
# 1. Khởi tạo LLM (Siêu nhanh và miễn phí từ Groq)
# Dùng model "llama-3.1-8b-instant"
llm = ChatGroq(
    model= model_llm,
    # temperature là tham số điều chỉnh độ sáng tạo của câu trả lời
    temperature=0 # Gần như = 0 để câu trả lời bám sát context
)
print(f"✅ Đã khởi tạo LLM (Groq {model_llm}).")

# 2. Khởi tạo Embedding Model (Dùng lại model cũ)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("✅ Đã khởi tạo Embedding Model (multilingual-MiniLM).")

✅ Đã cấu hình biến môi trường GROQ_API_KEY từ file .env.
✅ Đã khởi tạo LLM (Groq llama-3.1-8b-instant).
✅ Đã khởi tạo Embedding Model (multilingual-MiniLM).


## 3. Tải Vector Store

In [43]:
# Tải lại database đã được xây dựng và LỌC SẠCH ở file 01
db_path = "./chroma_db"
collection_name = "academic_regulations" # Đảm bảo tên này khớp với file 01

vectorstore = Chroma(
    persist_directory=db_path,
    embedding_function=embedding_model,
    collection_name=collection_name
)

# Tạo một "retriever" để lấy tài liệu
# search_kwargs={"k": 5}: Lấy 5 chunk liên quan nhất
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

print(f"✅ Đã tải thành công ChromaDB từ '{db_path}' (collection: '{collection_name}')")
print(f"📚 Số lượng chunks trong DB: {vectorstore._collection.count()}")

✅ Đã tải thành công ChromaDB từ './chroma_db' (collection: 'academic_regulations')
📚 Số lượng chunks trong DB: 143


## 4. Tạo Chuỗi RAG Hoàn chỉnh (End-to-End)

In [44]:
# 1. Tạo prompt RAG chính
# Ra lệnh cho LLM trả lời DỰA TRÊN context được cung cấp
rag_template = """
Bạn là một trợ lý AI hữu ích, chuyên trả lời các câu hỏi về quy định học thuật của trường 
ĐH Sư Phạm Kỹ Thuật TP.HCM dựa trên các văn bản được cung cấp.
Hãy trả lời câu hỏi của người dùng một cách ngắn gọn và chính xác, 
chỉ dựa vào nội dung trong phần "Văn bản tham khảo" dưới đây.
KHÔNG được bịa đặt thông tin. Nếu không tìm thấy, hãy nói "Tôi không tìm thấy thông tin này trong tài liệu."

Văn bản tham khảo:
{context}

Câu hỏi:
{question}

Câu trả lời (chỉ dựa trên văn bản):
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

original_context=""  # Biến toàn cục để lưu context sau khi gộp
# 2. Hàm này để gộp các chunk tìm được thành 1 đoạn text
def format_context(docs):
    global original_context
    context_parts = []
    
    for i, doc in enumerate(docs):
        # Lấy tên file từ metadata (chuẩn hóa dấu \ cho đẹp)
        source_file = doc.metadata.get('source', 'Không rõ nguồn').replace('\\', '/')
        
        # Tạo một khối context có đánh số và ghi nguồn
        part = f"""
--- NGUỒN THAM KHẢO #{i+1} ---
Nguồn file: {source_file}
Nội dung:
{doc.page_content}
"""
        context_parts.append(part)
    
    # Gộp tất cả lại
    original_context = "\n".join(context_parts)
    return original_context

# 3. TẠO CHUỖI RAG HOÀN CHỈNH
# Đây là "công thức" của RAG:
# 1. Lấy câu hỏi gốc (RunnablePassthrough)
# 2. Dùng câu gốc để tìm context (retriever)
# 3. Gộp context lại (format_context) và lưu vào biến "context"
# 4. Nhồi context và câu hỏi gốc vào RAG prompt (rag_prompt)
# 5. Đưa cho LLM trả lời (llm)
# 6. Lấy kết quả (StrOutputParser)

rag_chain = (
    {
        "context": RunnablePassthrough() 
                   | retriever 
                   | format_context,
        "question": RunnablePassthrough()
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("✅ Đã tạo chuỗi RAG hoàn chỉnh.")

✅ Đã tạo chuỗi RAG hoàn chỉnh.


## 5. Chạy Thử nghiệm Toàn bộ Pipeline

In [45]:
# Đặt câu hỏi (viết tắt)
# user_query = "thời gian đkmh là khi nào?"
user_query = "Mức điểm các loại chứng chỉ ngoại ngữ được sử dụng để xét miễn học các phần Kỹ năng giao tiếp tiếng Anh?"
print(f"🔍 Đang thực hiện truy vấn RAG cho câu hỏi: '{user_query}'")
print("\n" + "="*80)

# Dùng .stream() để xem LLM "gõ" câu trả lời
full_response = ""

for chunk in rag_chain.stream(user_query):
    print(chunk, end="", flush=True)
    full_response += chunk

print("\n" + "="*80)
print("\n✅ Hoàn tất!")

🔍 Đang thực hiện truy vấn RAG cho câu hỏi: 'Mức điểm các loại chứng chỉ ngoại ngữ được sử dụng để xét miễn học các phần Kỹ năng giao tiếp tiếng Anh?'

Tôi không tìm thấy thông tin về mức điểm các loại chứng chỉ ngoại ngữ được sử dụng để xét miễn học các phần Kỹ năng giao tiếp tiếng Anh trong các văn bản tham khảo.

✅ Hoàn tất!


In [46]:
print(f"Context gửi đi ({len(original_context)} ký tự) có số lượng tokens tối đa là: {llm.get_num_tokens(original_context)} tokens")
print(f"Nội dung context gửi đi:\n{original_context}")

Context gửi đi (5584 ký tự) có số lượng tokens tối đa là: 4691 tokens
Nội dung context gửi đi:

--- NGUỒN THAM KHẢO #1 ---
Nguồn file: ./data/1238 qd ban hanh quy dinh cac hoc phan ngoai ngu trong chuong trinh dao tao trinh do DH tu khoa 2023.pdf
Nội dung:
l c. Trong 01 tuần, trước thời gian kết thúc đóng học phí đợt 2 của mỗi học kỳ 06 tuân.

Ghi chú:

+ Tại thời điểm sinh viên nộp chứng chỉ ngoại ngữ để quy đổi SaHig điểm học phân kỹ năng giao tiếp tiếng Anh nếu điểm chứng chỉ ngoại ngữ đạt mức điểm chuẩn dâu ra tiếng Anh theo quy định hiện hành thì sẽ được xét đại chuẩn đâu ra ngoại ngữ khi xót tốt nghiệp (chứng chỉ phải còn hiệu lực tại thời điểm nộp);

____ + Sinh viên chỉ được sử dụng chứng chỉ quy đổi sang các học phân Kỹ năng giao tiếp tiếng Anh trong 07 học kỳ dâu tiên.

3. Quy định tổ chức các lớp học phần Kỹ năng giao tiếp tiếng Anh

- CELL tỏ chức kiểm tra phân loại chuẩn năng lực tiếng Anh đầu vào khi sinh viên đăng ký tham gia học phần Kỹ năng giao tiếp tiếng Anh 1. Những